In [1]:
import bootstrap  # noqa
import pandas as pd
import numpy as np
from ml_bias_explainability.explain_bias import ExplainBias
from ml_bias_explainability.bias_analysis import BiasAnalysis
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
columns_to_remove=[
    # output column
    # "two_year_recid",
    # duplicate columns
    "decile_score.1",
    "priors_count.1",
    # Not relevant
    "name",
    "first",
    "last",
    "compas_screening_date",
    "days_b_screening_arrest",
    "c_jail_in",
    "c_jail_out",
    "c_case_number",
    "c_offense_date",
    "c_arrest_date",
    "c_days_from_compas",
    "type_of_assessment",
    "screening_date",
    "v_type_of_assessment",
    "v_screening_date",
    "in_custody",
    "out_custody",
    # too similar with other features
    "dob",
    # too similar with output feature
    "decile_score",
    "is_recid",
    "r_case_number",
    "r_charge_degree",
    "r_days_from_arrest",
    "r_offense_date",
    "r_charge_desc",
    "r_jail_in",
    "r_jail_out",
    "violent_recid",
    "is_violent_recid",
    "vr_case_number",
    "vr_charge_degree",
    "vr_offense_date",
    "vr_charge_desc",
    "score_text",
    "v_decile_score",
    "v_score_text",
    "start",
    "end",
    "event",
]


In [5]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 20

def read_data():
        # The dataset used is taken from https://github.com/propublica/compas-analysis
        df = pd.read_csv("data/compas-scores-two-years.csv", index_col=0)

        # Filter certain rows based on propublica analysis
        df = df[df.days_b_screening_arrest <= 30]
        df = df[df.days_b_screening_arrest >= -30]
        df = df[df.is_recid != -1]
        df = df[df.c_charge_degree != "O"]
        df = df[df.score_text != "N/A"]

        # get rid of columns that shouldn't be in the input model
        df = df.drop(columns=columns_to_remove)

        # Ensure validity of inputs – No NaNs
        df = df.fillna(0)

        return df

output_column = "two_year_recid"
df = read_data()
df

,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,two_year_recid
id,,,,,,,,,,,
1,Male,69,Greater than 45,Other,0,0,0,0,F,Aggravated Assault w/Firearm,0
3,Male,34,25 - 45,African-American,0,0,0,0,F,Felony Battery w/Prior Convict,1
4,Male,24,Less than 25,African-American,0,0,1,4,F,Possession of Cocaine,1
7,Male,44,25 - 45,Other,0,0,0,0,M,Battery,0
8,Male,41,25 - 45,Caucasian,0,0,0,14,F,Possession Burglary Tools,1
...,...,...,...,...,...,...,...,...,...,...,...
10996,Male,23,Less than 25,African-American,0,0,0,0,F,Deliver Cannabis,0
10997,Male,23,Less than 25,African-American,0,0,0,0,F,Leaving the Scene of Accident,0
10999,Male,57,Greater than 45,Other,0,0,0,0,F,Aggravated Battery / Pregnant,0


In [7]:
# # Check for correlations between features to know which ones to remove from dataframe 
# # We don't want ones that are too similar to the output feature, i.e. a proxy of the output

# correlation_matrix = df.corr().abs()

# relevant_correlations = (
#     correlation_matrix.where(
#         np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool)
#     )
#     .stack()
#     .sort_values(ascending=False)
# )

# with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):  # more options can be specified also
#     print(relevant_correlations)

In [6]:
# Convert data for DL model
def convert_df_to_model_input(df, columns_to_remove, output_column):
    # extract output feature
    y_dataset = to_categorical(df[output_column])
    
    # get rid of columns that shouldn't be in the input model
#     df = df.drop(columns=columns_to_remove)
    
    # Convert df to an x_train tensorflow-input format 
    # numeric followed by categorical columns
    numeric_list = df.select_dtypes(include=[np.number]).columns
    df[numeric_list] = df[numeric_list].astype(np.float32)

    object_list = df.select_dtypes(object).columns
    for column in object_list:
        df[column] = pd.Categorical(df[column])
        df[column] = df[column].cat.codes

    # extract input features, replace3 NAN with -1
    x_dataset = to_categorical(np.where(np.isnan(df.values), -1, df.values))
    
    # flatted array into one dimension per sample
    x_dataset = x_dataset.reshape((x_dataset.shape[0], -1)) 
    
    return x_dataset, y_dataset

output_column = "two_year_recid"
x_dataset, y_dataset = convert_df_to_model_input(df, columns_to_remove, output_column)
x_dataset

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
# Split data into several arrays
total_length = len(x_dataset)
sixty_percent = round(total_length*0.6)
eighty_percent = round(total_length*0.8)

x_train = x_dataset[:sixty_percent]
x_val = x_dataset[sixty_percent+1:eighty_percent]
x_test = x_dataset[eighty_percent+1:]

y_train = y_dataset[:sixty_percent]
y_val = y_dataset[sixty_percent+1:eighty_percent]
y_test = y_dataset[eighty_percent+1:]

In [8]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [72]:
# Build the network
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(len(x_train[0]),)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))


# compile the network
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

# 4
history = model.fit(x_train, y_train, epochs=4, batch_size=512, validation_data=(x_val, y_val))
# history = model.fit(x_train, y_train, epochs=10, batch_size=512, validation_data=(x_val, y_val))

results = model.evaluate(x_test, y_test)

Epoch 1/4
8/8 [==============================] - 1s 35ms/step - loss: 0.6826 - acc: 0.5128 - val_loss: 0.6517 - val_acc: 0.5389
Epoch 2/4
8/8 [==============================] - 0s 6ms/step - loss: 0.6402 - acc: 0.5621 - val_loss: 0.6194 - val_acc: 0.6053
Epoch 3/4
8/8 [==============================] - 0s 6ms/step - loss: 0.6032 - acc: 0.6819 - val_loss: 0.5776 - val_acc: 0.7869
Epoch 4/4
39/39 [==============================] - 0s 625us/step - loss: 0.5404 - acc: 0.8264


In [73]:
results

[0.5403616428375244, 0.8264395594596863]

In [12]:
predictions = model.predict(x_test)
for prediction in predictions:
    print(round(prediction[0], 2), round(prediction[1], 2))

0.53 0.47
0.63 0.37
0.63 0.37
0.68 0.32
0.54 0.46
0.57 0.43
0.6 0.4
0.66 0.34
0.49 0.51
0.58 0.42
0.64 0.36
0.58 0.42
0.59 0.41
0.64 0.36
0.61 0.39
0.52 0.48
0.65 0.35
0.65 0.35
0.66 0.34
0.66 0.34
0.53 0.47
0.49 0.51
0.52 0.48
0.59 0.41
0.65 0.35
0.51 0.49
0.71 0.29
0.5 0.5
0.6 0.4
0.67 0.33
0.55 0.45
0.66 0.34
0.53 0.47
0.56 0.44
0.68 0.32
0.63 0.37
0.53 0.47
0.58 0.42
0.56 0.44
0.63 0.37
0.66 0.34
0.52 0.48
0.57 0.43
0.64 0.36
0.57 0.43
0.64 0.36
0.64 0.36
0.68 0.32
0.53 0.47
0.69 0.31
0.51 0.49
0.51 0.49
0.55 0.45
0.63 0.37
0.59 0.41
0.56 0.44
0.52 0.48
0.69 0.31
0.5 0.5
0.52 0.48
0.63 0.37
0.6 0.4
0.67 0.33
0.62 0.38
0.53 0.47
0.63 0.37
0.68 0.32
0.51 0.49
0.63 0.37
0.73 0.27
0.53 0.47
0.5 0.5
0.54 0.46
0.5 0.5
0.62 0.38
0.65 0.35
0.51 0.49
0.59 0.41
0.51 0.49
0.54 0.46
0.51 0.49
0.64 0.36
0.68 0.32
0.63 0.37
0.66 0.34
0.52 0.48
0.52 0.48
0.63 0.37
0.58 0.42
0.64 0.36
0.65 0.35
0.7 0.3
0.5 0.5
0.52 0.48
0.51 0.49
0.66 0.34
0.52 0.48
0.71 0.29
0.52 0.48
0.61 0.39
0.57 0.43
0.56 0.4

0.67 0.33
0.52 0.48
0.67 0.33
0.6 0.4
0.69 0.31
0.54 0.46
0.54 0.46
0.61 0.39
0.51 0.49
0.66 0.34
0.58 0.42
0.49 0.51
0.62 0.38
0.54 0.46
0.62 0.38
0.51 0.49
0.5 0.5
0.63 0.37
0.62 0.38
0.54 0.46
0.56 0.44
0.53 0.47
0.67 0.33
0.52 0.48
0.52 0.48
0.71 0.29
0.6 0.4
0.56 0.44
0.56 0.44
0.51 0.49
0.51 0.49
0.65 0.35
0.61 0.39
0.51 0.49
0.58 0.42
0.66 0.34
0.53 0.47
0.65 0.35
0.57 0.43
0.63 0.37
0.69 0.31
0.7 0.3
0.52 0.48
0.53 0.47
0.49 0.51
0.59 0.41
0.61 0.39
0.51 0.49
0.59 0.41
0.6 0.4
0.57 0.43
0.56 0.44
0.61 0.39
0.56 0.44
0.65 0.35
0.5 0.5
0.5 0.5
0.58 0.42
0.61 0.39
0.56 0.44
0.58 0.42
0.52 0.48
0.51 0.49
0.63 0.37
0.55 0.45
0.53 0.47
0.53 0.47
0.55 0.45
0.64 0.36
0.68 0.32
0.56 0.44
0.57 0.43
0.67 0.33
0.58 0.42
0.59 0.41
0.52 0.48
0.55 0.45
0.5 0.5
0.57 0.43
0.5 0.5
0.54 0.46
0.54 0.46
0.59 0.41
0.67 0.33
0.59 0.41
0.57 0.43
0.55 0.45
0.55 0.45
0.65 0.35
0.53 0.47
0.61 0.39
0.6 0.4
0.49 0.51
0.52 0.48
0.62 0.38
0.66 0.34
0.55 0.45
0.67 0.33
0.61 0.39
0.54 0.46
0.64 0.36
0.49 0.51


In [48]:
new_prediction = np.array([0.48, 0.52])
old_prediction = np.array([0.52, 0.48])

prediction_difference = new_prediction - old_prediction
print(prediction_difference)

std = np.std(prediction_difference)
mean = np.mean(prediction_difference)

print(std)
print(mean)

# std/mean

sum((prediction_difference - mean) / std)

[-0.04  0.04]
0.040000000000000036
0.0


0.0

In [53]:
old_prediction = model.predict(x_test)
old_prediction

array([[0.5285661 , 0.4714338 ],
       [0.6277289 , 0.3722711 ],
       [0.62586445, 0.37413546],
       ...,
       [0.64191854, 0.3580815 ],
       [0.5827341 , 0.41726595],
       [0.53372526, 0.4662747 ]], dtype=float32)

In [74]:
new_prediction = model.predict(x_test)
new_prediction

array([[0.49637026, 0.50362974],
       [0.63519156, 0.36480844],
       [0.6940631 , 0.3059369 ],
       ...,
       [0.6900633 , 0.3099367 ],
       [0.6020439 , 0.3979561 ],
       [0.49627852, 0.5037215 ]], dtype=float32)

In [77]:
prediction_difference = new_prediction - old_prediction
print(prediction_difference)

std = np.std(prediction_difference)
mean = np.mean(prediction_difference)

print(std)
# print(mean)


[[-0.03219587  0.03219596]
 [ 0.00746268 -0.00746265]
 [ 0.06819868 -0.06819856]
 ...
 [ 0.04814476 -0.04814479]
 [ 0.01930982 -0.01930985]
 [-0.03744674  0.03744677]]
0.04246777


In [90]:
np.mean(np.std(prediction_difference, axis=1))

0.036413886

In [92]:
np.sum(np.mean(prediction_difference, axis=1))

-2.3841858e-07

In [89]:
np.std(prediction_difference, axis=1)

array([0.03219591, 0.00746267, 0.06819862, ..., 0.04814477, 0.01930983,
       0.03744675], dtype=float32)

In [94]:
np.std(prediction_difference[:2], axis=1)

array([[-0.03219587,  0.03219596],
       [ 0.00746268, -0.00746265]], dtype=float32)

In [95]:
np.std(prediction_difference[:2], axis=1)

array([0.03219591, 0.00746267], dtype=float32)

In [96]:
np.mean(np.std(prediction_difference[:2], axis=1))

0.019829288

In [98]:
np.argmax(new_prediction[0])

1

In [99]:
new_prediction[0]

array([0.49637026, 0.50362974], dtype=float32)

In [101]:
old_prediction[0][np.argmax(new_prediction[0])]

0.4714338

In [114]:
(
            df.groupby(["race"])[[
                "age", "sex"
            ]]
            .mean()
            .reset_index()
        )

,race,age,sex
0,0,32.434959,0.827087
1,1,38.225807,0.935484
2,2,37.490726,0.770804
3,3,35.019646,0.838900
4,4,33.181820,0.818182
5,5,34.833820,0.830904


In [157]:
df = pd.DataFrame([[1,2,"yes"],[1,5,"k"],[1,8,"no"]])

In [158]:
df

,0,1,2
0,1,2,yes
1,1,5,k
2,1,8,no


In [159]:
(
            df.groupby([0])
#             .mean()
#             .agg({1:'sum', 2:'max'})
    .agg(mod = (2, lambda x: x.value_counts().index[0]),
         hello = (1, 'mean')
        )
            .reset_index()
        )

,0,mod,hello
0,1,k,5


In [126]:
np.mean(df[2])

TypeError: Could not convert yesnoyes to numeric

In [ ]:
# ExplainBias().check_probability_of_outcomes(model, x_test, y_test, 0.1)
# tensorboard, history.history

In [ ]:
unique_values_threshold = 10
verbose = False
predictions_df, aggregate_predictions_df = ExplainBias().modify_input_and_observe_effect_on_output(model, x_test, df, columns_to_remove, unique_values_threshold, verbose)
